#FaceMask detection model

**Build your CNN model using Keras to identify whether a person is wearing a faceMask or not**

###**1**.   **The Setup phase**


In [5]:
#---------------------------The import statments-----------------------------------------

import pandas as pd                               #reading, writing and manipulating the data (using tables)
import numpy as np                                #Library for linear algebra and some probabiltity (raw data) 
import tensorflow as tf                           #library for numerical computation that makes machine learning faster and easier
from tensorflow import keras                      #we may not add this since keras is already thier in tensorflow
from tensorflow.keras.models import Sequential    #To create the sequential layer
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout  #To create the model
from tensorflow.keras.optimizers import Adam      #Adam optimizer
from keras.preprocessing import image             #used for image classification
from keras.preprocessing.image import ImageDataGenerator  #used to expand the training dataset in order to improve the performance and ability of the model to generalize
import matplotlib.pyplot as plt                   #library to plot graphs
from google.colab import files                    #To be able to upload files

Here I will upload the dataset file called `"data"` that has 2 subfolders `"with"` and `"without`" and unzip




In [ ]:
#upload file
uploaded = files.upload()

In [ ]:
#unzipping the folder
!unzip data.zip

In [ ]:
#unzipping the folder
!unzip test-70x70.zip

In [ ]:
#delete the zip file as it is not needed anymore
!rm data.zip

In [ ]:
!rm test-70x70.zip

In [ ]:
# setting the batch size and the epochs

batch_size = 8
epochs = 30

Splitting the images (80% training and 20% 
validation) and Data augmanting it



In [ ]:


directory = 'data'

img_datagen = ImageDataGenerator(validation_split=0.2,        # Splits the data into training (80%) and validation (20%)
                                   rescale = 1./255,            # Multiple the colors by a number between 0-1 to process data faster
                                   rotation_range=40,           #rotate the images
                                   width_shift_range=0.2,       #fraction of the total width
                                   height_shift_range = 0.2,    #fraction of the total height
                                   zoom_range = 0.2,            #float percent
                                   horizontal_flip=True,        #horizontal flip
                                   fill_mode='nearest')        #add new pixels when the image is rotated or shifted

train_generator = img_datagen.flow_from_directory(
                                directory,
                                target_size = (70, 70),
                                batch_size = batch_size,
                                color_mode="rgb",               # for coloured images
                                class_mode = 'binary',
                                shuffle = True,
                                seed=42,                      # to make the result reproducible
                                subset = 'training')            # Specify this is training set

validation_generator = img_datagen.flow_from_directory(
                                directory,
                                target_size = (70, 70),
                                batch_size = batch_size,
                                color_mode="rgb",               # for coloured images
                                class_mode = 'binary',
                                subset = 'validation')            # Specify this is training set



**Display a batch of the images used in the training and thier labels**

In [ ]:
#generate a batch of images and labels from the training set
imgs, labels = next(train_generator)

In [ ]:
#plotting function

def plotImages(images_arr):
    fig, axes = plt.subplots(1, batch_size, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
#displaying the images and thier labels where as 0 with mask and 1 without mask
plotImages(imgs);
print(labels);





###**2. Build and train the CNN**





Sequencial is a list of the layers of the model we want to create. Here it consists of

*   Conv2D Layer
> * **The filter** parameter means the number of this layer's output filters which is less in the early layers and more when we are closer to the prediction, [recommended to start up with 32,64,128 and the number varies according to the depth of the model]
> *   **The kernal_size** specifies the width and the height of the 2D convolution window [odd integer and depend on the image size if image size > 128x128 then use 5*5 if less use 3x3 or 1x1]
> *   **The activation** parameter refers to the type of activation function
> *   **The padding** parameter is enabled to zero-padding to preserve the spatial dimensions of the volume so the output volume size matches the input volume size
> *   **The input_shape** parameter has pixel high and pixel wide and have the 3 color channels: RGB

*   MaxPool2D Layer
> To pool and reduce the dimensionlaity of the data
> *   pool_size: max value over a 2x2 pooling window
> *  strides: how far the pooling window moves for each pooling step
*   Flatten Layer
> * flatten is used to flatten the input to a 1D vector then passed to dense

*   Dense Layer (The output layer)
> * **The units** parameter means that it has 2 nodes one for with and one for without because we want a binary output 
> *   **The activation** parameter we use the softmax activation function on our output so that the output for each sample is a probability distribution over the outputs of with and without mask






In [ ]:
#create the model layers
#we have functional and sequential
model = Sequential([
                    Conv2D(filters=32, kernel_size=(3,3),activation='relu',padding='same',input_shape=(70,70,3)),
                    MaxPool2D(pool_size=(2,2), strides=2),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding= 'same'),
                    MaxPool2D(pool_size=(2,2), strides =2),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding= 'same'),
                    MaxPool2D(pool_size=(2,2), strides =2),
                    Flatten(),
                    Dense(units=64, activation= 'relu'),
                    #means the output is 0,1 (the labels) and the P(c=0) +P(c=1) = 1 
                    Dense(units=1, activation='sigmoid'), 

])

#check out the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 70, 70, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 35, 35, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

Compile the model using the **Adam** optimizer with **learning rate** of `0.0001`, a **loss** of `binary_crossentropy`, and we'll look at `accuracy` as our performance **metric**.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),         # remsprop, adadelta
              loss='binary_crossentropy',                   # mean_squared_error (regression task), categorical_crossentropy
              metrics=['accuracy'])                         #metrices to keep track off

We use the `train_generator` because we are now only training the data. The 
validation data is the `validation_generator` 


In [ ]:
#Training the model
history = model.fit(train_generator ,epochs = epochs,validation_data= validation_generator, batch_size= batch_size) 

Epoch 1/30
1002/1002 [==============================] - 51s 18ms/step - loss: 0.5149 - accuracy: 0.7309 - val_loss: 0.3887 - val_accuracy: 0.8277
Epoch 2/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.2986 - accuracy: 0.8770 - val_loss: 0.3997 - val_accuracy: 0.8242
Epoch 3/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.2558 - accuracy: 0.8947 - val_loss: 0.2897 - val_accuracy: 0.8816
Epoch 4/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.2233 - accuracy: 0.9134 - val_loss: 0.2569 - val_accuracy: 0.8966
Epoch 5/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.1874 - accuracy: 0.9311 - val_loss: 0.2857 - val_accuracy: 0.8856
Epoch 6/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.1716 - accuracy: 0.9350 - val_loss: 0.3619 - val_accuracy: 0.8641
Epoch 7/30
1002/1002 [==============================] - 18s 18ms/step - loss: 0.1551 - accuracy: 0.9403 - val_loss: 0.1934 -

In [ ]:
model.save('MaskCheck.model')

In [ ]:
new_model = tf.keras.models.load_model('MaskCheck.model') #loading a model



### **3. Plotting the loss and accuracy of training vs validation**






In [ ]:
#Plotting the loss of validation and training 
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochstoplot = range(1,epochs+1)
plt.plot(epochstoplot, loss_train, 'g', label='Training loss')
plt.plot(epochstoplot, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [2]:
#Plotting the accuracy of validation and training 
accur_train = history.history['accuracy']
accur_val = history.history['val_accuracy']
plt.plot(epochstoplot, accur_train, 'g', label='Training accuracy')
plt.plot(epochstoplot, accur_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: ignored

### **4. Testing the CNN model**


In [ ]:
from IPython.display import Image, display
TGREEN =  '\033[1;37;42m'
TRED =    '\033[1;37;41m'
for i in range (1,17):
  img_directory = str(i) + '.jpg'
  img_data = image.load_img(img_directory, target_size = (70, 70))   #load the image from the directory
  img_data = image.img_to_array(img_data)                            #convert the image to a Numpy array
  img_data = np.expand_dims(img_data, axis = 0)                     #expands the array by inserting a new axis at the specified position.

  classify = model.predict(img_data)
  display(Image(img_directory,width= 150, height=150))
  print("\n")
  if(int(classify[0][0]) == 0):
    print(TGREEN + "The person is wearing a mask. \n")
  else:
    print(TRED + "The person is not wearing a mask.\n")